In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd

import duckdb
def query(s):
    return duckdb.sql(s).df()

books_metadata = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "hadifariborzi/amazon-books-dataset-20k-books-727k-reviews",
  "amazon_books_metadata_sample_20k.csv",
)

books_reviews = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "hadifariborzi/amazon-books-dataset-20k-books-727k-reviews",
  "amazon_books_metadata_sample_20k.csv",
)



In [ ]:
# (Optional) Save the file locally 
import os

outdir = './dataset'
if not os.path.exists(outdir):
    os.mkdir(outdir)

books_metadata.to_csv('./dataset/metadata.csv',)
books_reviews.to_csv('./dataset/reviews.csv')

In [8]:
samplebook = query(
    """    
    select
        title, 
        publisher_date, 
        author_name,
        cast(trim(right(publisher_date,4)) as int64) published_year, 
        parent_asin 
    from books_metadata 
    where 1=1 
        and lower(author_name) like '%suzanne collins%'
    order by published_year desc
        limit 100
    """
)
samplebook.head()

,title,publisher_date,author_name,published_year,parent_asin
0,The Hunger Games: The Special Edition (Hunger ...,"Special Edition (October 30, 2018",Suzanne Collins,2018,1338321919
1,The Hunger Games: Official Illustrated Movie C...,"Original edition (February 7, 2012",Suzanne Collins,2012,0545422906
2,Hunger Games 3 - La revolte [ en grand format ...,"0 edition (May 5, 2011",Suzanne Collins,2011,2266182714
3,The Hunger Games Trilogy Boxed Set,"August 24, 2010",Suzanne Collins,2010,0545265355
4,The Hunger Games (Book 1),"Reprint edition (July 3, 2010",Suzanne Collins,2010,0439023521


In [ ]:
p1 = query(
    """
select
      *
from books_reviews
where parent_asin in (select distinct parent_asin from samplebook)
-- limit 5      
"""
)

query("""
select parent_asin, count(1) from p1 group by 1 order by 2 desc
      """)

,parent_asin,count(1)
0,0439023521,3150
1,0545265355,1827
2,0545422906,77
3,043979143X,17
4,2266182714,1
5,1338321919,1
